In [1]:
import os
import random
import shutil
import tqdm
import numpy as np
import pandas as pd
import copy
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import *
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.activations import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras import layers, optimizers

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

/home/shanmukh.alle/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shanmukh.alle/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shanmukh.alle/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shanmukh.alle/miniconda3/lib/py

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

In [3]:
sampling_rate = 100
path = "../../dataset/processed100/force/"
# path = "../dataset/processed50/lp_residual/"
# path = "./dataset/raw/"
modelstore = "/scratch/shanmukh.alle/models"

In [4]:
walk_df = pd.read_csv("../../dataset/WalksDemographics.csv")
walk_df

,Walk Name,Patient ID,Gender,Age,Height,Weight,HoehnYahr,UPDRS,UPDRSM,TUAG,Speed,Extra Task,Class
0,GaCo01_01.txt,GaCo01,1,66,1.80,83.0,0.0,0.0,0.0,NaN,1.075,0,0
1,GaCo02_01.txt,GaCo02,1,74,1.74,70.0,0.0,1.0,1.0,NaN,1.040,0,0
2,GaCo02_02.txt,GaCo02,1,74,1.74,70.0,0.0,1.0,1.0,NaN,1.162,0,0
3,GaCo03_01.txt,GaCo03,1,69,1.80,101.0,0.0,0.0,0.0,NaN,1.051,0,0
4,GaCo03_02.txt,GaCo03,1,69,1.80,101.0,0.0,0.0,0.0,NaN,1.265,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,SiPt36_01.txt,SiPt36,2,53,1.58,62.0,2.0,52.0,32.0,11.27,0.970,0,1
302,SiPt37_01.txt,SiPt37,2,66,1.70,62.0,2.5,27.0,21.0,7.56,1.010,0,1
303,SiPt38_01.txt,SiPt38,2,65,1.59,60.0,2.0,22.0,14.0,10.13,1.070,0,1
304,SiPt39_01.txt,SiPt39,2,69,1.68,53.0,2.0,33.0,20.0,13.97,0.880,0,1


In [5]:
data = {}
for i in set(walk_df["Patient ID"]):
    data[i] = []
    
for index,row in tqdm.tqdm(walk_df.iterrows()):
    patient = row["Patient ID"]
#     features = row[["Gender","Age","Height","Weight","TUAG","Speed","Extra Task"]]
    features = row[["Gender","Age","Height","Weight","TUAG"]]

    walk_name = row["Walk Name"]
    walk_seq = np.loadtxt(path+walk_name,delimiter=",")
    sample = (walk_seq,features,row["Class"])
    data[patient].append(sample)


306it [00:42,  7.18it/s]


In [6]:
walk_seq.shape

(12119, 18)

In [7]:
patients = sorted(list(data.keys()))
labels = []
for i in patients:
    labels.append(walk_df[walk_df["Patient ID"]==i].iloc[0]["Class"])
#     print (i,labels[-1])

In [8]:
# Helper Functions

def window(samples,feature,label,cut_length=100):
    inputs = []
    features = []
    labels = []
    
    for i in range(len(samples)):
        sample = samples[i]
        cut = int(cut_length/2)
        for j in range(int(len(sample)/cut)):
            if (j+2)*cut>=len(sample):
                break
            inputs.append(sample[j*cut:(j+2)*cut,:])
            features.append(feature[i])
            labels.append(label[i])
            
    inputs = np.stack(inputs)
    features = np.array(features)
    labels = np.array(labels)
    
    return inputs, features, labels

def pad(samples):
    lengths = [len(i) for i in samples]
    max_len = max(lengths)
    for i in range(len(samples)):
        pad_len = max_len - lengths[i]
        samples[i] = np.pad(samples[i],((0,pad_len),(0,0)),"wrap")
    return np.stack(samples)
    
def get_from_dict(dictionary, keys):
    output = []
    for i in keys:
        output += dictionary[i]
    return output

def get_best_model(path):
    models = os.listdir(path)
    accuracy = {}
    for i in models:
        info = i.split("-")
        try:
            accuracy[float(info[-1][:-5])][float(info[-2])] = i
        except:
            accuracy[float(info[-1][:-5])] = {}
            accuracy[float(info[-1][:-5])][float(info[-2])] = i
    best_acc = max(accuracy.keys())
    best_loss = min(accuracy[best_acc].keys())
    model_path = accuracy[best_acc][best_loss]
    
    model = tf.keras.models.load_model(path+"/"+model_path)
    return model

class VariancePooling(tf.keras.layers.Layer):
    def __init__(self, ):
        super(VariancePooling, self).__init__()

    def call(self, x):
        return tf.math.reduce_std(x,axis=1)

In [16]:
def conv1D_full():
    '''
    :return: 1 branch of the parallel Convnet
    '''
    input1 = Input(shape=(100, 1))
    x = Conv1D(filters=8, kernel_size=3, activation='selu', padding='valid')(input1)
    x = Conv1D(filters=16, kernel_size=3, activation='selu', padding='valid')(x)
    x = layers.MaxPooling1D(2)(x)
    x = Conv1D(filters=16, kernel_size=3, activation='selu', padding='valid')(x)
    x = Conv1D(filters=16, kernel_size=3, activation='selu', padding='valid')(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(50, activation='elu')(x)
    model = Model(input1, x)
    rms = optimizers.RMSprop(lr=0.001, decay=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
#     print(model.summary())
    return model


def multiple_cnn1D(nb=18):
    '''
    :param nb: number of features ( indicates the number of parallel branches)
    :return:
    '''
    inputs = Input(shape=(100,nb))
    outputs = []
    for i in range(nb):
        x = inputs[:,:,i]
        model = conv1D_full()
        x = tf.expand_dims(x,axis=-1)
        x = model(x)
        outputs.append(x)
    x = concatenate(outputs,axis=-1)
    x = Dropout(0.5)(x)
    x = layers.Dense(100, activation='selu')(x)
    x =  Dropout(0.5)(x)
    x = layers.Dense(20, activation='selu')(x)
    x = Dropout(0.5)(x)
    answer = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs, answer)
    opt = optimizers.RMSprop(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
#     print(model.summary())
    return model

def baseline(np=18):
        # Define Model
    input_layer = Input(shape=(None,18))
    x = SpatialDropout1D(0.2)(input_layer)
    x = Conv1D(32,7,activation="linear")(x)
    x = BatchNormalization()(x)
    x = Activation("elu")(x)
    x = MaxPool1D(2,2)(x)
    x = Conv1D(32,5,activation="linear")(x)
    x = BatchNormalization()(x)
    x = Activation("elu")(x)
    x = MaxPool1D(2,2)(x)
    x = Conv1D(64,3,activation="linear")(x)
    x = BatchNormalization()(x)
    x = Activation("elu",name="embedding")(x)
    avg_pool = GlobalAvgPool1D()(x)
    avg_pool = Dropout(0.25)(avg_pool)
    
    prediction = Dense(1,activation="sigmoid",name="det")(avg_pool)
    model = Model(inputs=input_layer, outputs=prediction)
    
    model.compile(optimizer=keras.optimizers.Adam(lr=0.0001),
          loss=keras.losses.BinaryCrossentropy(label_smoothing=0.1),
          metrics=['accuracy'])
    return model

In [18]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=3)
gold = []
predictions = []
AUC = []
accuracy = []
F1Score = []

fold = 0
# try:
#     shutil.rmtree(modelstore)
# except:
#     pass

for trainI,testI in kfold.split(patients,labels):
    fold+=1
    train_patients = [patients[i] for i in trainI]
    test_patients = [patients[i] for i in testI]
    print (test_patients)
    
    
    # Get Walk level data
    train = get_from_dict(data,train_patients)
    test = get_from_dict(data,test_patients)
    
    trainSeq = [i[0] for i in train]
    trainNum = [i[1].values for i in train]
    trainLabel = [i[2] for i in train]

    testSeq = [i[0] for i in test]
    testNum = [i[1].values for i in test]
    testLabel = [i[2] for i in test]
    
    # Normalize Numerical data
    scaler = StandardScaler()
    trainNum = scaler.fit_transform(trainNum)
    testNum = scaler.transform(testNum)
    
    # Impute Numerical data
    imputer = KNNImputer()
    trainNum = imputer.fit_transform(trainNum)
    testNum = imputer.transform(testNum)
    
    # Get Windowed Version of Input
#     trainSeqWindows, trainNumWindows, trainLabelWindows = window(trainSeq,trainNum,trainLabel) 
    trainSeqWindows = []
    valSeqWindows = []
    
    trainLabelWindows = []
    valLabelWindows = []
    for i in range(len(trainSeq)):
        tempwindows,_,templabels = window([trainSeq[i]],[trainNum[i]],[trainLabel[i]])
        trainwins,valwins,trainlab,vallab = train_test_split(tempwindows,templabels,test_size=0.1,random_state=0)
        trainSeqWindows.append(trainwins)
        trainLabelWindows.append(trainlab)
        
        valSeqWindows.append(valwins)
        valLabelWindows.append(vallab)
        
        
    trainSeqWindows = np.concatenate(trainSeqWindows,axis=0)
    trainLabelWindows = np.concatenate(trainLabelWindows,axis=0)
    
    valSeqWindows = np.concatenate(valSeqWindows,axis=0)
    valLabelWindows = np.concatenate(valLabelWindows,axis=0)
        
    testSeqWindows, testNumWindows, testLabelWindows = window(testSeq,testNum,testLabel) 

    
    
    # Define Model
    full_model = baseline()
    earlystopper = EarlyStopping(monitor="val_loss",mode="min",patience=20, restore_best_weights=True,verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor="val_loss",factor=0.5,mode="min",patience=15,verbose=1,min_lr=0.0000001)
    
    full_model.fit(trainSeqWindows,trainLabelWindows,batch_size=800,epochs=1000,
              validation_data=[valSeqWindows,valLabelWindows],
              callbacks=[earlystopper,reduce_lr],verbose=1)
    

    print("Train Performance \n")
    full_model.evaluate(trainSeqWindows,trainLabelWindows)
    print("Val Performance \n")
    full_model.evaluate(valSeqWindows,valLabelWindows)
    print("Test Performance \n")
    full_model.evaluate(testSeqWindows,testLabelWindows)

    break

['GaCo02', 'GaCo05', 'GaCo06', 'GaCo15', 'GaCo16', 'GaCo17', 'GaPt05', 'GaPt06', 'GaPt15', 'GaPt24', 'GaPt25', 'GaPt32', 'JuCo11', 'JuCo24', 'JuPt03', 'JuPt05', 'JuPt06', 'JuPt10', 'JuPt16', 'JuPt20', 'SiCo05', 'SiCo07', 'SiCo09', 'SiCo14', 'SiCo23', 'SiCo27', 'SiCo29', 'SiPt04', 'SiPt10', 'SiPt15', 'SiPt16', 'SiPt20', 'SiPt35']
Train on 45853 samples, validate on 5277 samples
Epoch 1/1000
45853/45853 [==============================] - 2s 38us/sample - loss: 0.6427 - acc: 0.6462 - val_loss: 0.6086 - val_acc: 0.7224
Epoch 2/1000
45853/45853 [==============================] - 1s 15us/sample - loss: 0.5997 - acc: 0.7033 - val_loss: 0.5766 - val_acc: 0.7391
Epoch 3/1000
45853/45853 [==============================] - 1s 15us/sample - loss: 0.5716 - acc: 0.7329 - val_loss: 0.5505 - val_acc: 0.7845
Epoch 4/1000
45853/45853 [==============================] - 1s 15us/sample - loss: 0.5520 - acc: 0.7526 - val_loss: 0.5342 - val_acc: 0.8164
Epoch 5/1000
45853/45853 [==============================

In [15]:
for i in trainSeqWindows:
    print (i.shape)

(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)
(100, 18)


In [12]:
np.concatenate(trainSeqWindows,axis=0).shape


(4969900, 18)